Use an `ImageFolder` dataset to load the data, and pass it through the network we used earlier. Use a `Resize` transform before the `ToTensor` transform to convert the data to a uniform resolution of 28x28 and pass it through the network of the previous section. See
what kind of performance you can achieve.

Note: The `Resize` transform will turn our variable resolution data into a default (low) resolution. Whether the images are 32x32, 48x48 or 64x64, they all end up at 28x28 before the network sees them. This is not ideal, since we are losing a lot of information, but it is cheap, because all our images are very small. We'll use this network as a simple baseline later in the assignment (at question 17).

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision.transforms import *
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

import numpy as np

In [32]:
TRAIN_SIZE = 50000
VALIDATION_SIZE = 10000
ROOT = '../data' # .. to not have data folder within notebooks folder
BATCH_SIZE = 64
EPOCHS = 3

LEARNING_RATE = 0.001

In [33]:
transform = transforms.Compose([transforms.Resize([28, 28]),
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

In [36]:
dataset = torchvision.datasets.ImageFolder(root = "../data/mnist-varres/train/",
                                          transform=transform)

train_set, validation_set = torch.utils.data.random_split(dataset, [TRAIN_SIZE, VALIDATION_SIZE])

train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           num_workers = 2)

validation_loader = torch.utils.data.DataLoader(validation_set,
                                                batch_size = BATCH_SIZE,
                                                shuffle = True,
                                                num_workers = 2)

In [37]:
test_set = torchvision.datasets.ImageFolder(root = "../data/mnist-varres/test/",
                                          transform=transform)

test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size = BATCH_SIZE,
                                          shuffle = True,
                                          num_workers = 2)

In [38]:
log_interval = 100

In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1) # size: batch, 1, 28, 28
        self.conv2 = nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1) # (batch, 16, 14, 14)
        self.conv3 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1) # (batch, 32, 7, 7)
        self.fc1 = nn.Linear(64*3*3, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) # (batch, 16, 28, 28) # TODO klopt het dat relu en maxpool zijn omgedraaid?
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # (batch, 32, 14, 14)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2) # (batch, 64, 7, 7)
        x = torch.flatten(x, 1) # (batch, 64, 3, 3)
        x = self.fc1(x)
        return F.log_softmax(x, dim = 1)

In [46]:
network = Net()
optimizer = optim.Adam(network.parameters(), lr=LEARNING_RATE)

In [47]:
train_losses = []
train_counter = []
val_losses = []
val_counter = [i*TRAIN_SIZE for i in range(EPOCHS + 1)]

In [48]:
# TODO Make sure to have "results" directory in "CNN/"
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(network.state_dict(), '../results/model_Q8.pth')
            torch.save(optimizer.state_dict(), '../results/optimizer_Q8.pth')

In [49]:
def test():
    network.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            output = network(data)
            val_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        val_loss /= VALIDATION_SIZE
        val_losses.append(val_loss)
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            val_loss, correct, VALIDATION_SIZE,
            100. * correct / VALIDATION_SIZE))

In [50]:
test()
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test()

/Users/jannekehulsen/miniconda3/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3073, Accuracy: 1029/10000 (10%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.326840
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.457407
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.961121
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.720492
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.403678
Train Epoch: 1 [32000/50000 (64%)]	Loss: 0.312039
Train Epoch: 1 [38400/50000 (77%)]	Loss: 0.483754
Train Epoch: 1 [44800/50000 (90%)]	Loss: 0.177289

Test set: Avg. loss: 0.3313, Accuracy: 9064/10000 (91%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.394373
Train Epoch: 2 [6400/50000 (13%)]	Loss: 0.228322
Train Epoch: 2 [12800/50000 (26%)]	Loss: 0.184118
Train Epoch: 2 [19200/50000 (38%)]	Loss: 0.407885
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.391542
Train Epoch: 2 [32000/50000 (64%)]	Loss: 0.310461
Train Epoch: 2 [38400/50000 (77%)]	Loss: 0.283297
Train Epoch: 2 [44800/50000 (90%)]	Loss: 0.148738

Test set: Avg. loss: 0.2342, Accuracy: 9285/10000 (93%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 0.